In [4]:
date = 20220831
nYear = 365

#LDAY = 30 #June

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

#Loans
location_loan = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\01_MIB_Loans\\"+str(date)+"\\MIB_Loans"+str(date)+".txt"
loan = pd.read_csv(location_loan, sep = "\t", header = 0, low_memory = False)
loan.columns = loan.columns.str.replace(" ", "_")

Product_Hierarchy = pd.read_csv (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\06_Product_Hierarchy\Product_Hierarchy.txt", sep = "\t", header = 0, low_memory = False)
Product_Hierarchy.columns = Product_Hierarchy.columns.str.replace(" ", "_")

PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc','Level_6','Level_5']]
PRD = PRD.sort_values(by=['Product_Hierarchy_Cd'], ascending=True)

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

loan.M_Product_Hierarchy_Cd = loan.M_Product_Hierarchy_Cd.astype(str)
loan.M_Product_Hierarchy_Cd = loan.M_Product_Hierarchy_Cd.str.upper()
loan.M_Product_Hierarchy_Cd = loan.M_Product_Hierarchy_Cd.str.strip()

loan001 = loan.merge(PRD.rename(columns={'Product_Hierarchy_Cd': 'M_Product_Hierarchy_Cd'}), on = 'M_Product_Hierarchy_Cd', how = 'left')


In [13]:
LN1 = loan001.iloc[np.where((loan001.M_Acct_Status_Cd==1)&(loan001.M_Full_Writeoff_Flg=='N'))]


LN1['LEVEL_6_ORI'] = LN1['Level_6']
LN1['DATA_SOURCE'] = '01. LOAN'
#LN1['SECURITY_TYPE_CD'] = -9999
LN1.loc[(LN1['Level_6']=='Unit Trust')&(LN1['Source_Prod_Type_Dsc'].str.contains('ASB2')),'Level_6'] = 'Unit Trust-ASB2'
LN1.loc[(LN1['Level_6']=='Unit Trust')&(LN1['Source_Prod_Type_Dsc'].str.contains('ASB')),'Level_6'] = 'Unit Trust-ASB'
LN1.loc[LN1['Level_6'].isin(['Unit Trust','Housing','Shophouse','Other Mortgage LN1', 'Salary']),'Level_6'] = 'Others'


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [14]:
def PRODUCT_GROUP_APC(DATA_SOURCE,LEVEL_6,M_Sub_Market_Segment_Cd,M_Product_Hierarchy_Cd):
    #NON-RETAIL FINANCING
    if( ((DATA_SOURCE!='07. HP')&(DATA_SOURCE!='08. CC'))&\
       ((DATA_SOURCE == '01. LOAN')|(DATA_SOURCE == '02. OD')|(DATA_SOURCE == '03. STRC')|(DATA_SOURCE == '04. INV')|\
        (DATA_SOURCE == '05. TRADE')|(DATA_SOURCE == '06. FCL'))&((LEVEL_6!='Salary')&(LEVEL_6!='SMF'))):
        if((M_Sub_Market_Segment_Cd == 5)|(M_Sub_Market_Segment_Cd == 6)|(M_Sub_Market_Segment_Cd == 7)|\
           (M_Sub_Market_Segment_Cd == 8))|((M_Product_Hierarchy_Cd=='HZK01')|(M_Product_Hierarchy_Cd=='HZK02')|\
                                        (M_Product_Hierarchy_Cd=='HZK03')|(M_Product_Hierarchy_Cd=='HZK04')):
            return '01. Corporate Banking'
        elif(M_Sub_Market_Segment_Cd == 2):
            return '02. Business Banking'
        elif(M_Sub_Market_Segment_Cd == 1):
            return '03. RSME Banking'
    #RETAIL FINANCING
    #MORTGAGE FINANCING
    if(((DATA_SOURCE == '01. LOAN')|(DATA_SOURCE == '02. OD')|(DATA_SOURCE == '06. FCL'))&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|\
                                                                                           (M_Sub_Market_Segment_Cd == 4))):
        if(LEVEL_6 == 'Housing'):
            return '04. Home Financing'
        elif(LEVEL_6 == 'Shophouse'):
            return '05. Shophouse'
        elif(LEVEL_6 == 'Other Mortgage Loan'):
            return '06. Other Mortgage'
    #HIRE PURCHASE
    if(DATA_SOURCE == '07. HP'):
        return '07. Hire Purchase'
    #PRODUCT EXCLUDE FROM PERSONAL FINANCING ONLY WO CARDS
    if((DATA_SOURCE == '01. LOAN')&((M_Product_Hierarchy_Cd == '711')|(M_Product_Hierarchy_Cd == '714'))):
        return '08. Credit Cards'
    if(DATA_SOURCE == '08. CC'):
        return '08. Credit Cards'
    #UNIT TRUST
    if((DATA_SOURCE == '01. LOAN')&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))):
        if((LEVEL_6 == 'Unit Trust')|(LEVEL_6 == 'Unit Trust-ASB')|(LEVEL_6 == 'Unit Trust-ASB2')):
            return '09. Unit Trust'
    #SALARY
    if(LEVEL_6=='Salary'):
        return '11. Salary'
    #OTHER RETAIL FINANCING
    if((DATA_SOURCE == '01. LOAN')&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))):
        if(((LEVEL_6!='Unit Trust')&(LEVEL_6!='Unit Trust-ASB')&(LEVEL_6!='Unit Trust-ASB2')&\
            (LEVEL_6!='Housing')&(LEVEL_6!='Shophouse')&(LEVEL_6!='Other Mortgage Loan')&(LEVEL_6!='Salary'))&\
           (DATA_SOURCE == '01. LOAN')&((M_Product_Hierarchy_Cd!='711')&(M_Product_Hierarchy_Cd!='714')&\
                                        (M_Product_Hierarchy_Cd!='HZK01')&(M_Product_Hierarchy_Cd!='HZK02')&\
                                        (M_Product_Hierarchy_Cd!='HZK03')&(M_Product_Hierarchy_Cd!='HZK04'))):
            return '12. Other Retail Financing'
    #CASHLINE
    if((DATA_SOURCE=='02. OD')&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))&\
       ((LEVEL_6!='SMF')&(LEVEL_6!='Housing')&(LEVEL_6!='Shophouse')&(LEVEL_6!='Other Mortgage Loan'))):
        return '13. Cashline'
    #SMF
    if(LEVEL_6=='SMF'):
        return '14. SMF'
LN1['PRODUCT_GROUP_APC'] = LN1.apply(lambda x: PRODUCT_GROUP_APC(x['DATA_SOURCE'], x['Level_6'],x['M_Sub_Market_Segment_Cd'],x['M_Product_Hierarchy_Cd']), axis=1)


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
#/*************************** SURIA*****************************************/


LOAN_SURIA = LN1.iloc[np.where(LN1.M_Account_No.isin([452022604015\
,461033839807\
,451575301708\
,451249110716]))].rename(columns={'M_Bnm_Balance_SUM':'M_BNM_BALANCE'})

LOAN_SURIA['LEVEL_6_ORI'] = LOAN_SURIA.Level_6

#LOAN_SURIA=LOAN_SURIA[['M_Account_No'\
#,'M_Cus_No'\
#,'M_BNM_BALANCE'\
#,'Ead_Proportional_Myr'\
#,'Rwa_After_Risk_Mit_Myr'\
#,'Expected_Loss_Myr'\
#,'LEVEL_6_ORI'\
#,'M_Sub_Market_Segment_Cd'\
#,'M_Product_Hierarchy_Cd'\
#,'Source_Prod_Type_Dsc'\
#,'Approach'\
#,'Sector_Category']]

In [21]:
writer = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\12_Investment_Account\\'+str(date)+'\\IA_LOAN'+str(date)+'(python).xlsx',engine='xlsxwriter')


LN1.to_excel(writer, sheet_name='IA_LOAN'+str(date), index = False)

LOAN_SURIA.to_excel(writer, sheet_name='LOAN_SURIA'+str(date), index = False)

writer.save()


In [15]:
LN1.head(1)

,As_Of_Date,M_Account_No,M_Cus_No,M_Source_Prod_Type_Cd,M_Approved_Limit_SUM,M_Bnm_Balance_SUM,M_Product_Hierarchy_Cd,M_Economic_Sector_Cd,Branch_Code,M_Date_Aa_Approved,Amrt_Term,M_Date_First_Released,M_Date_Final_Released,M_Npl_Date,M_Mths_Installment_In_Arrears,M_Reschedule_Flg,M_Restructure_Flg,M_Interest_Income_SUM,M_Non_Interest_Income_SUM,Cur_Gross_Rate_SUM,M_Penalty_Charges_SUM,M_Misc_Charges_SUM,M_Funds_Charge_Myr_SUM,M_Interest_Income_Myr_SUM,M_Net_Int_Income_Myr_SUM,M_Special_Mention_Acct_Flg,M_New_Npl_Flg,M_Entity_Cd,M_Acct_Status_Cd,M_Npl_Flg,Avg_Book_Bal_SUM,Adjustable_Type_Cd,Maturity_Date,M_Full_Writeoff_Flg,M_Interest_Rate_Type,M_Interest_Code,M_Sub_Market_Segment_Cd,M_Aa_Number,Amrt_Term_Mult,M_Financing_Concept,Source_Prod_Type_Dsc,Level_6,Level_5,LEVEL_6_ORI,DATA_SOURCE,PRODUCT_GROUP_APC
0,31-AUG-2022,630010000209,23444900,HZK03,652260.0,652260.0,HZK03,0310,1401,21-SEP-2020,60,04-FEB-2021,NaN,01-JAN-1900,0,NaN,NaN,1917.7,0.0,3.8,0.0,0.0,1570.79,1917.7,346.91,N,N,MIB,1,N,652260.0,250,08-MAR-2026,N,S,46,3.0,GMD20200083,M,B03,Rent-To-Own v3,RTO,Leasing,RTO,01. LOAN,01. Corporate Banking


In [17]:
LN1.M_Account_No.dtypes

dtype('int64')

In [16]:
#with RDMS
#LN1.loc[LN1.Level_6 == 'Unit Trust-ASB', 'P2_RWA_MIL'] = (LN1.EAD_PROPORTIONAL_MYR - LN1.RWA_AFTER_RISK_MIT_MYR)/1000000
#LN1.loc[LN1.Level_6 != 'Unit Trust-ASB', 'P2_RWA_MIL'] = 0

In [25]:
LN1.PRODUCT_GROUP_APC.value_counts()

09. Unit Trust                390918
04. Home Financing            238777
03. RSME Banking               46969
11. Salary                     40677
05. Shophouse                  13055
08. Credit Cards                9561
06. Other Mortgage              6573
01. Corporate Banking           5639
02. Business Banking            4052
12. Other Retail Financing       326
Name: PRODUCT_GROUP_APC, dtype: int64

In [27]:
LN1.iloc[np.where(LN1.PRODUCT_GROUP_APC=='09. Unit Trust')]

,As_Of_Date,M_Account_No,M_Cus_No,M_Source_Prod_Type_Cd,M_Approved_Limit_SUM,M_Bnm_Balance_SUM,M_Product_Hierarchy_Cd,M_Economic_Sector_Cd,Branch_Code,M_Date_Aa_Approved,Amrt_Term,M_Date_First_Released,M_Date_Final_Released,M_Npl_Date,M_Mths_Installment_In_Arrears,M_Reschedule_Flg,M_Restructure_Flg,M_Interest_Income_SUM,M_Non_Interest_Income_SUM,Cur_Gross_Rate_SUM,M_Penalty_Charges_SUM,M_Misc_Charges_SUM,M_Funds_Charge_Myr_SUM,M_Interest_Income_Myr_SUM,M_Net_Int_Income_Myr_SUM,M_Special_Mention_Acct_Flg,M_New_Npl_Flg,M_Entity_Cd,M_Acct_Status_Cd,M_Npl_Flg,Avg_Book_Bal_SUM,Adjustable_Type_Cd,Maturity_Date,M_Full_Writeoff_Flg,M_Interest_Rate_Type,M_Interest_Code,M_Sub_Market_Segment_Cd,M_Aa_Number,Amrt_Term_Mult,M_Financing_Concept,Source_Prod_Type_Dsc,Level_6,Level_5,DATA_SOURCE,PRODUCT_GROUP_APC
21,31-AUG-2022,451016002913,17045132,U1,10204.0,6353.31,U1,0131,5101,07-FEB-2011,20,21-FEB-2011,21-FEB-2011,01-JAN-1900,2,N,N,25.09,0.0,4.60,5.0,0.0,15.08,25.09,10.01,Y,N,MIB,1,N,6368.76,250,21-FEB-2031,N,M,34,3.0,2011F7763,Y,A02,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
22,31-AUG-2022,451016003508,11095181,U1,101148.0,20559.05,U1,0131,5101,17-MAR-2011,14,29-MAR-2011,29-MAR-2011,01-JAN-1900,0,N,N,73.44,0.0,4.25,0.0,0.0,48.67,73.44,24.77,N,N,MIB,1,N,20559.05,250,29-MAR-2025,N,S,34,3.0,2011F13786,Y,A02,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
29,31-AUG-2022,451016009542,21613756,U1,10262.0,8392.40,U1,0131,5101,29-OCT-2013,25,31-OCT-2013,31-OCT-2013,01-JAN-1900,0,N,N,30.50,0.0,4.60,0.0,0.0,21.11,30.50,9.39,N,N,MIB,1,N,8392.40,250,31-OCT-2038,N,M,34,3.0,2013F43348,Y,A02,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
31,31-AUG-2022,451016009781,13786809,U1,31325.0,24587.91,U1,0131,5101,08-NOV-2013,25,18-NOV-2013,18-NOV-2013,01-JAN-1900,0,N,N,86.76,0.0,4.25,0.0,0.0,61.84,86.76,24.92,N,N,MIB,1,N,24587.91,250,18-NOV-2038,N,S,34,3.0,2013F44661,Y,A02,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
32,31-AUG-2022,451016010351,18388487,U1,51404.0,41982.00,U1,0131,5101,19-NOV-2013,25,28-NOV-2013,28-NOV-2013,01-JAN-1900,0,N,N,152.04,0.0,4.25,0.0,0.0,105.59,152.04,46.45,N,N,MIB,1,N,41982.00,250,28-NOV-2038,N,S,34,3.0,2013F46101,Y,A02,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764973,31-AUG-2022,469972026700,5013716,U1,30000.0,29064.17,U1,0131,6997,28-FEB-2022,8,18-MAR-2022,18-MAR-2022,01-JAN-1900,0,N,N,96.82,0.0,3.90,0.0,0.0,63.81,96.82,33.01,N,N,MIB,1,N,28970.47,250,18-MAR-2030,N,M,46,3.0,2022E2533,Y,B06,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
764974,31-AUG-2022,469972026747,14000526,U1,31459.0,31240.99,U1,0131,6997,15-MAR-2022,20,22-MAR-2022,22-MAR-2022,01-JAN-1900,0,N,N,104.75,0.0,3.95,0.0,0.0,72.43,104.75,32.32,N,N,MIB,1,N,31139.62,250,22-MAR-2042,N,S,46,3.0,2022E2957,Y,B06,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
764979,31-AUG-2022,469972027422,21787801,U1,62331.0,62301.03,U1,0131,6997,18-MAY-2022,25,30-MAY-2022,30-MAY-2022,01-JAN-1900,0,N,N,208.72,0.0,3.95,0.0,0.0,149.55,208.72,59.17,N,N,MIB,1,N,62099.04,250,30-MAY-2047,N,S,46,3.0,2022E5464,Y,B06,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
764981,31-AUG-2022,469972027499,17258375,U1,81312.0,81393.17,U1,0131,6997,25-MAY-2022,35,31-MAY-2022,31-MAY-2022,01-JAN-1900,0,N,N,265.58,0.0,3.85,0.0,0.0,195.45,265.58,70.13,N,N,MIB,1,N,81159.00,250,31-MAY-2057,N,M,46,3.0,2022E5785,Y,B06,ASB FINANCING - I,Unit Trust,Unit Trust,01. LOAN,09. Unit Trust
